In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gauravneupane/flavia-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flavia-dataset


In [3]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [3]:
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.utils import to_categorical
import os


def load_flavia_dataset_from_csv(csv_path, img_size, base_image_dir=None):
    """
    Load and preprocess Flavia dataset from a CSV file with robust error handling

    Parameters:
    -----------
    csv_path : str
        Path to the CSV file containing image paths and labels
    img_size : tuple
        Desired image size (width, height)
    base_image_dir : str, optional
        Base directory for image files if paths in CSV are relative

    Returns:
    --------
    X : numpy array
        Preprocessed image data
    y : numpy array
        Integer labels
    y_onehot : numpy array
        One-hot encoded labels
    class_names : list
        List of unique class names
    """
    # Read the CSV file
    df = pd.read_csv(csv_path)

    X = []  # Images
    y = []  # Labels

    # Ensure we have the correct columns
    if 'id' not in df.columns or 'y' not in df.columns:
        raise ValueError("CSV must contain 'image path' and 'y' columns")

    # Track unique classes for naming
    class_names = df['y'].unique().tolist()

    # Iterate through rows in the dataframe
    for _, row in df.iterrows():
        # Construct full path if base_image_dir is provided
        img_path = row['id']
        if base_image_dir:
            img_path = os.path.join(base_image_dir, img_path)

        try:
            # Check if file exists
            if not os.path.exists(img_path):
                print(f"File not found: {img_path}")
                continue

            # Open image with PIL
            with Image.open(img_path) as img:
                # Convert to RGB if not already
                img = img.convert('RGB')

                # Resize image
                img = img.resize(img_size, Image.LANCZOS)

                # Convert to numpy array and normalize
                img_array = np.array(img) / 255.0

                X.append(img_array)
                y.append(row['y'])

        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue

    # Check if any images were loaded
    if len(X) == 0:
        raise ValueError("No images could be loaded. Check file paths and image formats.")

    # Convert to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # One-hot encode labels
    num_classes = len(np.unique(y))
    y_onehot = to_categorical(y, num_classes=num_classes)

    print(f"Loaded {len(X)} images")
    return X, y, y_onehot, class_names

In [4]:
X,y,y_onehot, class_names = load_flavia_dataset_from_csv(csv_path ='/kaggle/input/flavia-dataset/Leaves/all.csv', img_size=(256,256), base_image_dir='/kaggle/input/flavia-dataset/Leaves')

Loaded 1907 images


In [5]:
X.shape

(1907, 256, 256, 3)

In [6]:
 # Split data into training + validation set and test set (e.g., 80% train/val, 20% test)
X_trainval, X_test, y_trainval, y_test = train_test_split(
 X, y_onehot, test_size=0.2, random_state=42, stratify=y_onehot
     )

# Split the training + validation set into training and validation sets (e.g., 80% train, 20% val from the 80% train/val set)
X_train, X_val, y_train, y_val = train_test_split(
 X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval
     )

In [7]:
datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )

In [8]:
def create_xception_transfer_learning_model(input_shape, num_classes):
    """
    Create a transfer learning model using Xception as the base

    Parameters:
    -----------
    input_shape : tuple
        Shape of input images (height, width, channels)
    num_classes : int
        Number of classes to classify

    Returns:
    --------
    Model : tensorflow.keras.Model
        Compiled transfer learning model
    """

    # Load pre-trained Xception model
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze base model layers
    base_model.trainable = False

    # Add custom classification layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=output)

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [9]:
 # Create and compile the model
model = create_xception_transfer_learning_model(
        input_shape=X.shape[1:],
        num_classes=y_onehot.shape[1]
    )

    # Callbacks
reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )

early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )

    # Train the model
history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        epochs=30,
        validation_data=(X_val, y_val),
        steps_per_epoch=len(X_train) // 32,
        callbacks=[reduce_lr, early_stopping],
        verbose=1
    )

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 361s 9s/step - accuracy: 0.0900 - loss: 3.3995 - val_accuracy: 0.5836 - val_loss: 2.6229 - learning_rate: 1.0000e-04
Epoch 2/30
 1/38 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.5000 - loss: 2.8098

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


38/38 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.5000 - loss: 2.8098 - val_accuracy: 0.5836 - val_loss: 2.5992 - learning_rate: 1.0000e-04
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 344s 9s/step - accuracy: 0.3779 - loss: 2.6870 - val_accuracy: 0.7016 - val_loss: 1.7991 - learning_rate: 1.0000e-04
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.5625 - loss: 2.1765 - val_accuracy: 0.7115 - val_loss: 1.7781 - learning_rate: 1.0000e-04
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 345s 9s/step - accuracy: 0.5460 - loss: 2.0357 - val_accuracy: 0.7934 - val_loss: 1.2021 - learning_rate: 1.0000e-04
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.6875 - loss: 1.6649 - val_accuracy: 0.7836 - val_loss: 1.1904 - learning_rate: 1.0000e-04
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 341s 9s/step - accuracy: 0.6182 - loss: 1.5356 - val_accuracy: 0.8590 - val_loss: 0.8341 - learning_rate: 1.0000e-04
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.7188 - loss: 1.1771 

In [10]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluate the model's performance on the test set

    Parameters:
    -----------
    model : tensorflow.keras.Model
        Trained model
    X_test : numpy array
        Test images
    y_test : numpy array
        One-hot encoded test labels

    Returns:
    --------
    Prints model evaluation metrics
    """
    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"\nFinal Test Loss: {test_loss:.4f}")
    print(f"Final Test Accuracy: {test_accuracy * 100:.2f}%")

    # Predict and generate a classification report
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    from sklearn.metrics import classification_report
    print("\nDetailed Classification Report:")
    print(classification_report(y_true_classes, y_pred_classes))

In [11]:
    # Evaluate the model
evaluate_model(model, X_test, y_test)

    # Optional: Save the model
model.save('xception_leaf_classification_model.keras')
print("Model saved successfully.")

12/12 ━━━━━━━━━━━━━━━━━━━━ 83s 7s/step - accuracy: 0.9145 - loss: 0.3650

Final Test Loss: 0.3313
Final Test Accuracy: 92.15%
12/12 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step

Detailed Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        12
           1       0.91      0.77      0.83        13
           2       0.93      1.00      0.96        13
           3       1.00      1.00      1.00        15
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00        11
           6       0.90      0.75      0.82        12
           7       0.91      1.00      0.95        10
           8       0.82      0.82      0.82        11
           9       0.92      0.92      0.92        12
          10       1.00      0.80      0.89        10
          11       1.00      0.92      0.96        13
          12       1.00      0.80      0.89        10
          13       0.92      0.92      0.92  

In [3]:
from tensorflow.keras.models import load_model

model = load_model("model/")

2025-08-12 08:44:16.736660: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
model.layers

[<InputLayer name=input_layer, built=True>,
 <Conv2D name=block1_conv1, built=True>,
 <BatchNormalization name=block1_conv1_bn, built=True>,
 <Activation name=block1_conv1_act, built=True>,
 <Conv2D name=block1_conv2, built=True>,
 <BatchNormalization name=block1_conv2_bn, built=True>,
 <Activation name=block1_conv2_act, built=True>,
 <SeparableConv2D name=block2_sepconv1, built=True>,
 <BatchNormalization name=block2_sepconv1_bn, built=True>,
 <Activation name=block2_sepconv2_act, built=True>,
 <SeparableConv2D name=block2_sepconv2, built=True>,
 <BatchNormalization name=block2_sepconv2_bn, built=True>,
 <Conv2D name=conv2d, built=True>,
 <MaxPooling2D name=block2_pool, built=True>,
 <BatchNormalization name=batch_normalization, built=True>,
 <Add name=add, built=True>,
 <Activation name=block3_sepconv1_act, built=True>,
 <SeparableConv2D name=block3_sepconv1, built=True>,
 <BatchNormalization name=block3_sepconv1_bn, built=True>,
 <Activation name=block3_sepconv2_act, built=True>,
 <

In [ ]:
model.predict()